In [ ]:
from mmdet.apis import init_detector, inference_detector
import mmcv
import paths
import pickle
import os
from process_dataset import common

In [ ]:
# Epoch number (to choose a checkpoint)
e = 31

# device = "cuda"
device = "cpu"

score_thr = 0.5
# score_thr = 0

# Video input filename
input_filename = "day_hq.mp4"

output_id = input_filename.split(".")[0] + f"_e{e}_{score_thr}"

# Video output filename
output_filename = output_id + ".mp4"

# Step (every n-th frame will be taken, other ignored)
step = 5

# Frames output folder
frames_dirname = output_id

# Pick a specific .pth file
paths.last_checkpoint_filepath = os.path.join(paths.working_dirpath, f"epoch_{e}.pth")

# stop_at_frames = 0
stop_at_frames = 100

Error `UserWarning: Class names are not saved in the checkpoint's meta data, use COCO classes by default.`
can be ignored. Just now, even after receiving this error, images are annotated correctly with my classes.

In [ ]:
model = init_detector(paths.model_config_filepath, paths.last_checkpoint_filepath, device=device)
model.CLASSES = common.classes # Needed for correct class names

# I wanted to use the original yolox config and just adjust the number of 
# classes and classes themselves here, but this strangely results in misdetections
# when inferencing... So, in this script, it is necessary to use a custom config
# where the number of classes is updated
# model.bbox_head.num_classes = len(common.classes)

video = mmcv.VideoReader(input_filename)

frames_dir = os.path.join(paths.working_dirpath, frames_dirname)
if not os.path.exists(frames_dir):
    os.mkdir(frames_dir)

counter = 0
try:
    for frame in video:
        print(counter // step, end="\r")

        if counter % step == 0:
            result = inference_detector(model, frame)
            out_filepath = os.path.join(frames_dir, str(counter // step).zfill(6) + ".jpg")

            if score_thr == 0:
                model.show_result(frame, result, out_file=out_filepath)
            else:
                model.show_result(frame, result, score_thr=score_thr, out_file=out_filepath)

        counter += 1
        if counter == stop_at_frames * step: # Doesn't stop if stop_at_frames == 0
            raise KeyboardInterrupt

except KeyboardInterrupt:
    print("Stop requested")

In [ ]:
fps = 30 // step
mmcv.frames2video(frames_dir, os.path.join(paths.working_dirpath, output_filename), fps=fps)